# Evaluate GPT-3 and GPT-4 for Korean sentence completions

Inputs used here are formatted as one of the following.
- Nom-Nom: Subject1-Nom Object1-Acc V. Subject2-Nom Object1-Acc
- Nun-Nun: Subject1-Nun Object1-Acc V. Subject2-Nun Object1-Acc
- NC-Nom: Subject2-Nom Object1-Acc
- NC-Nun: Subject2-Nun Object1-Acc

Model inserts the continuation of the second partial sentence. Then it is evaluated whether the same verb as the "context verb" (the first verb in the prompt) is repeated or negated. 

In the Nun-Nun condition, it is pragmatically felicitious to continue the sentence with the contrasted (negated or different) predicate from the previous sentence.

In [1]:
#for generation
import os
import openai

import pandas as pd
from tqdm import tqdm 

In [2]:
#for analysis
import numpy as np
import pickle
import csv
import math
import ast

In [3]:
#for tokenizing Korean
from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()

In [4]:
# Supply your own OpenAI API keys
keys = open('/Users/hagyeongshin/Experiments/openai-keys.txt')
lines = keys.readlines()
openai.organization = lines[0].rstrip()
openai.api_key = lines[1].rstrip()

In [5]:
def insert_gpt3(item):
    """
    Generate sentence continuation with GPT-3. 
    Print out the output in a dictionary format.
    """
    
    output = openai.Completion.create(
        model = "text-davinci-003",
        prompt = item,
        suffix = ".",
        max_tokens=30, 
        temperature=0, # argmax
        n= 1,
        logprobs= 1,
        stop= [".","\n"]
        )
    
    output_dict = output.to_dict()['choices'][0].to_dict()
    
    return output_dict

In [6]:
def insert_gpt4(item):
    """
    Generate sentence continuation with GPT-4. 
    Print out the output in a dictionary format.
    """
    
    output = openai.ChatCompletion.create(
        model = "gpt-4-0613",
        messages = [{"role": "user", "content":item}],
        max_tokens=30,
        temperature=0, #argmax
        n= 1,
        stop= [".","\n"]
        )
    
    output_dict = output.to_dict()['choices'][0].to_dict()
    
    return output_dict

Only GPT-3 gives the log probability of inserted tokens as of now. We are seeing here whether the context verb is contrasted or not. 

cverb_pos[0] is a root of the context verb.
1) cverb_pos[0] + VX | VXV -> negated (=contrasted)
2) cverb_pos[0] w/o VX | VXV -> repeated (=not contrasted)
3) no cverb_pos[0] -> different (further annotation is needed to determine the relation)

In [7]:
def eval_relation(token_pos, cverb_pos):
    """
    Determine whether the model generated the same, 
    negated or different verb than the context verb. 
    
    'contrasted': context verb root with negation
    'repeated': context verb root
    'different': no context verb root
    """
    
    relation = ''
    
    for i in range(len(token_pos)):
        if cverb_pos[0] in token_pos and 'VX' in token_pos[i] or 'VXV' in token_pos[i]: 
            relation = 'contrasted'
            break
        elif cverb_pos[0] in token_pos and 'VX' not in token_pos[i] and 'VXV' not in token_pos[i]: 
            relation = 'repeated'
        else:    
            relation = 'different'
    return relation

In [8]:
def main_gpt3(filename):
    """
    Sentence continuations for each prompt given in a CSV format using GPT-3.
    Compile the output and POS tags of the created tokens into a dataframe.
    """
    
    data_path = "input/{FILE}.csv".format(FILE=filename)
    df_output = pd.read_csv(data_path)
    
    token = []
    tpos = []
    cpos = []
    rel = []
    
    with tqdm(total=df_output.shape[0]) as pbar:
        for index, row in df_output.iterrows():
        
            prompt = row['full_prompt']
            
            t = insert_gpt3(prompt)
            token.append(t['text'])
            
            token_pos = kkma.pos(t['text']) #Korean pos tagger 
            tpos.append(token_pos)
            
            cverb_pos = kkma.pos(row['context_verb'])
            cpos.append(cverb_pos)
            
            relation = eval_relation(token_pos, cverb_pos)
            rel.append(relation)
            
            pbar.update(1)
            
    df_output['token'] = token
    df_output['token_pos'] = tpos
    df_output['cverb_pos'] = cpos
    df_output['relation'] = rel

    df_output.to_csv("{TASK}_gpt3_output.csv".format(TASK=filename))
    
    return df_output

In [9]:
def main_gpt4(filename):
    """
    Sentence continuations for each prompt given in a CSV format using GPT-4.
    Compile the output and POS tags of the created tokens into a dataframe.
    """
    
    data_path = "input/{FILE}.csv".format(FILE=filename)
    df_output = pd.read_csv(data_path)
    
    token = []
    tpos = []
    cpos = []
    rel = []
    
    with tqdm(total=df_output.shape[0]) as pbar:
        for index, row in df_output.iterrows():
        
            prompt = row['full_prompt']
            
            t = insert_gpt4(prompt)
            token.append(t['message']['content'])
            
            token_pos = kkma.pos(t['message']['content']) #Korean pos tagger
            tpos.append(token_pos)
            
            cverb_pos = kkma.pos(row['context_verb'])
            cpos.append(cverb_pos)
            
            relation = eval_relation(token_pos, cverb_pos)
            rel.append(relation)
            
            pbar.update(1)
            
    df_output['token'] = token
    df_output['token_pos'] = tpos
    df_output['cverb_pos'] = cpos
    df_output['relation'] = rel

    df_output.to_csv("{TASK}_gpt4_output.csv".format(TASK=filename))
    
    return df_output

In [10]:
def main_gpt34(filename):
    """
    Sentence continuations for each prompt given in a CSV format using GPT-3 and GPT-4.
    The final dataframe compiles output from both GPT-3 and GPT-4 together a single dataframe.
    """
    
    data_path = "input/{FILE}.csv".format(FILE=filename)
    df_output = pd.read_csv(data_path)
    
    token_3 = []
    token_4 = []
    tpos_3 = []
    tpos_4 = []
    rel_3 = []
    rel_4 = []
    cpos = []
    
    with tqdm(total=df_output.shape[0]) as pbar:
        for index, row in df_output.iterrows():
        
            prompt = row['full_prompt']
            
            cverb_pos = kkma.pos(row['context_verb'])
            cpos.append(cverb_pos)
            
            t_3 = insert_gpt3(prompt)
            token_3.append(t_3['text'])
            
            token3_pos = kkma.pos(t_3['text'])
            tpos_3.append(token3_pos)
            
            relation_3 = eval_relation(token3_pos, cverb_pos)
            rel_3.append(relation_3)
            
            t_4 = insert_gpt4(prompt)
            token_4.append(t_4['message']['content'])
            
            token4_pos = kkma.pos(t_4['message']['content'])
            tpos_4.append(token4_pos)           
            
            relation_4 = eval_relation(token4_pos, cverb_pos)
            rel_4.append(relation_4)
            
            pbar.update(1)
            
    df_output['cverb_pos'] = cpos
    df_output['token_gpt3'] = token_3    
    df_output['token_gpt3_pos'] = tpos_3
    df_output['relation_gpt3'] = rel_3
    df_output['token_gpt4'] = token_4
    df_output['token_gpt4_pos'] = tpos_4
    df_output['relation_gpt4'] = rel_4

    df_output.to_csv("{TASK}_gpt34_output.csv".format(TASK=filename))
    
    return df_output

In [11]:
output = main_gpt34('subj-insert-sample')

100%|█████████████████████████████████████████████| 4/4 [00:14<00:00,  3.52s/it]


In [13]:
output = main_gpt34('subj-insert-full')

100%|█████████████████████████████████████████| 112/112 [05:32<00:00,  2.97s/it]


In [14]:
#output = pd.read_csv('subj-insert-full_gpt34_output.csv')

In [15]:
pd.set_option('display.max_rows', 500)
output

,item_id,scenario_id,condition,scenario,target,full_prompt,context_verb,cverb_pos,token_gpt3,token_gpt3_pos,relation_gpt3,token_gpt4,token_gpt4_pos,relation_gpt4
0,1,1,nc-nom,지은이 친구들 다섯 명과 함께 마라톤 경주에 참여했습니다. 지은은 다른 약속이 있어...,민찬이가 메달을,지은이 친구들 다섯 명과 함께 마라톤 경주에 참여했습니다. 지은은 다른 약속이 있어...,받다,"[(받, VV), (다, EFN)]","얻었고, 영훈이는 두 번째","[(얻, VV), (었, EPT), (고, ECE), (,, SP), (영, NNG...",different,"딴 것은 아니지만, 모두 잘 끝냈어","[(땋, VV), (ㄴ, ETD), (것, NNB), (은, JX), (아니, VC...",different
1,2,2,nc-nom,영준과 은지는 다른 세 명의 친구들과 함께 오후 5시에 만나기로 약속했습니다. 영준...,채영이가 집을,영준과 은지는 다른 세 명의 친구들과 함께 오후 5시에 만나기로 약속했습니다. 영준...,나오다,"[(나오, VV), (다, EFN)]","빠져서 늦게 왔고, 소연이","[(빠지, VV), (어서, ECD), (늦, VA), (게, ECD), (오, V...",different,"방금 나갔다고 했고, 지훈이는 버스를 타고 오고 있다고","[(방금, MAG), (나가, VV), (었, EPT), (다고, EFN), (하,...",contrasted
2,3,3,nc-nom,혜미와 혜미의 친구들이 연말 파티를 하기로 했습니다. 몇몇 친구들은 음식을 준비하고...,동준이가 음식을,혜미와 혜미의 친구들이 연말 파티를 하기로 했습니다. 몇몇 친구들은 음식을 준비하고...,만들다,"[(만들, VV), (다, EFN)]","준비했고, 지원이가 파","[(준비, NNG), (하, XSV), (었, EPT), (고, ECE), (,, ...",different,"준비했고, 지은이와 선영이가 파티 장소를 준비","[(준비, NNG), (하, XSV), (었, EPT), (고, ECE), (,, ...",different
3,4,4,nc-nom,각 지역 신문은 매일 일기 예보 기사를 전합니다. 민준이 아현에게 오늘의 일기 예보...,부산 신문이 비 소식을,각 지역 신문은 매일 일기 예보 기사를 전합니다. 민준이 아현에게 오늘의 일기 예보...,알리다,"[(알리, VV), (다, EFN)]",전하고 있습니다,"[(전하, VV), (고, ECE), (있, VXV), (습니다, EFN)]",contrasted,전하고 있어,"[(전하, VV), (고, ECE), (있, VV), (어, ECD)]",different
4,5,5,nc-nom,홍보팀장이 다섯 명의 팀원들로부터 전달받은 프로젝트 보고서를 연구팀에 보냈습니다. ...,박상엽 사원이 계획서를,홍보팀장이 다섯 명의 팀원들로부터 전달받은 프로젝트 보고서를 연구팀에 보냈습니다. ...,쓰다,"[(쓰, VV), (다, EFN)]","작성하고, 이승호 사원이","[(작성, NNG), (하, XSV), (고, ECE), (,, SP), (이승, ...",different,"작성하였고, 김지은 사원이 시장조사를 담당하였습니다","[(작성, NNG), (하, XSV), (였, EPT), (고, ECE), (,, ...",different
5,6,6,nc-nom,승우가 친구들과 함께 온라인 추리 게임을 했습니다. 승우는 범인이 누구라고 생각하는...,범수가 단서를,승우가 친구들과 함께 온라인 추리 게임을 했습니다. 승우는 범인이 누구라고 생각하는...,알다,"[(알, VV), (다, EFN)]",모두 찾아냈습니다,"[(모두, MAG), (찾아내, VV), (었, EPT), (습니다, EFN)]",different,모두 찾아서 게임을 끝냈습니다,"[(모두, MAG), (찾, VV), (아서, ECD), (게임, NNG), (을,...",different
6,7,7,nc-nom,민아와 상현이가 유기견을 돕는 봉사활동을 하고 있습니다. 민아는 교실 앞에 기부함을...,태성이가 먹을 것을,민아와 상현이가 유기견을 돕는 봉사활동을 하고 있습니다. 민아는 교실 앞에 기부함을...,주다,"[(주, VXV), (다, EFN)]","기부했고, 지연이가 놀이","[(기부, NNG), (하, XSV), (었, EPT), (고, ECE), (,, ...",different,기부했어,"[(기부, NNG), (하, XSV), (었, EPT), (어, EFN)]",different
7,8,8,nc-nom,상훈이 친구들과 함께 피구게임을 했고 혜미는 게임을 하지 않았습니다. 상훈은 게임 ...,건후가 공을,상훈이 친구들과 함께 피구게임을 했고 혜미는 게임을 하지 않았습니다. 상훈은 게임 ...,맞다,"[(맞닿, VV)]",,[],,잘 차서 결국에는 게임에서 이겼어,"[(잘, MAG), (차, VV), (아서, ECD), (결국, NNG), (에, ...",different
8,9,9,nc-nom,주희가 여러 명의 미술 작가들이 참여한 전시회에 구경을 갔지만 마감 시간이 되어서 ...,이강호 작가가 관심을,주희가 여러 명의 미술 작가들이 참여한 전시회에 구경을 갔지만 마감 시간이 되어서 ...,모으다,"[(모으, VV), (다, EFN)]",끌었어요,"[(끌, VV), (었, EPT), (어요, EFN)]",different,많이 받았어요,"[(많이, MAG), (받, VV), (았, EPT), (어요, EFN)]",different
9,10,10,nc-nom,윤지가 친구들의 연극 공연을 보러 갔습니다. 윤지는 공연에 늦게나마 도착했지만 도현...,은영이가 인기를,윤지가 친구들의 연극 공연을 보러 갔습니다. 윤지는 공연에 늦게나마 도착했지만 도현...,끌다,"[(끌, VV), (다, EFN)]",많이 받았어,"[(많이, MAG), (받, VV), (았, EPT), (어, EFN)]",different,많이 끌었어,"[(많이, MAG), (끌, VV), (었, EPT), (어, EFN)]",repeated
